In [235]:
!pip install bertopic[visulaization]

In [236]:
!pip install bertopic[visulaization]

In [237]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [238]:
emails = pd.read_csv('C:\Project DBs\Email\emails.csv')
#email_subset = emails[:10000]
email_subset = emails.sample(frac=0.02, random_state=1)

print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [239]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [240]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        
    }

In [241]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('');
            
        else:
            results.append(email[key])
    return results

In [242]:
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

def filter_words(texts):
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [243]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head(3))

                                                body
0  It would be nice if you could be at my dinner,...
1  Absolutely.Good point!  Can Peter start to dra...
2  My apologies.  My schedule melted down after w...


In [244]:
gmail_data=pd.read_csv('C:\\Users\\user\\Downloads\\MailBodyOutput2.csv')
gmail_data.dropna(inplace=True)
gmail_data.head()

,ID,body,to,from_
0,A1,"Hi Dmml,\r\nWelcome to Google. Your new accoun...","('dmml2test@gmail.com',)",googlecommunityteam-noreply@google.com
1,A2,"Your impartial, comprehensive news source.\r\n...","('dmml2test@gmail.com',)",DailyDigest@email.join1440.com
4,A5,** Important: Confirm Your Subscription to Nex...,"('dmml2test@gmail.com',)",dave@davenetics.com
5,A6,Your NextDraft subscription has been confirmed...,"('dmml2test@gmail.com',)",dave@davenetics.com
6,A7,Welcome to ELEVATOR.\r\n\r\nYour inbox is like...,"('dmml2test@gmail.com',)",yo@gettheelevator.com


In [245]:
email_df=email_df.append(gmail_data.drop('ID',axis=1))

In [246]:
docs = list(email_df.loc[:, "body"].values)

In [247]:
docs=filter_words(docs)

In [248]:
doc=[]
for x in docs:
    doc.append(' '.join(x))

In [249]:
doc[:5]

['would nice could dinner since probably knowanyone else anytime want go lunch check house status glad go',
 'absolutely good point peter start draft override letter kay mann enrondieball john grigby reminder us need form override letter go withthe form turbine contract kay',
 'apologies schedule melted talked monday swhere folks came concern size supposed beno larger lobbied aceves apparently ok oversized group folks group talked himoriginally pretty sure five violate rules folks wonderedif groups smaller could hook upwith sorry wrinkle think wespoke gets real ugly trying find smaller group let know fortunately another team case due two weeks best jeff',
 'vince uk var breached limit last week uk traders asked us review correlations across uk gas power aswell correlations across efa slots part work last week update correlations based historical prices tanya richard lewisleppard lon ect ect rudy dautel hou ect ect kirstee hewitt lon ect ect naveen andrews corp enron enron david port ma

In [250]:
model = BERTopic(language="english")

In [251]:
topics, probs = model.fit_transform(doc)

In [252]:
model.get_topic_freq()

,Topic,Count
0,-1,5844
1,0,486
2,1,265
3,2,237
4,3,205
...,...,...
141,140,11
142,141,11
143,142,10
144,143,10


In [253]:
model.get_topic(0)

[('enron', 0.029616467311789667),
 ('com', 0.02354997869175887),
 ('scott', 0.00919191890828519),
 ('mark', 0.00796312952786479),
 ('susan', 0.007509610218330219),
 ('david', 0.007122691157978628),
 ('john', 0.006994447064537558),
 ('paul', 0.005957165118725919),
 ('taylor', 0.005952141286282726),
 ('smith', 0.00556425144321082)]

In [254]:
model.get_topic(2)

[('going', 0.024008143986541646),
 ('good', 0.023891227945177607),
 ('weekend', 0.0177362270541082),
 ('get', 0.01748295383006711),
 ('go', 0.016814939391504538),
 ('hope', 0.015803381370434764),
 ('know', 0.015188479754216513),
 ('night', 0.014901469869714844),
 ('love', 0.013616276906697254),
 ('anyway', 0.013492760711536345)]

In [255]:
model.get_topic(5)

[('would', 0.010365940689266837),
 ('ect', 0.008458265876829421),
 ('rate', 0.007304221452508822),
 ('contract', 0.007301786608339348),
 ('enron', 0.0071762653400508575),
 ('market', 0.00689155936199737),
 ('price', 0.006585561145107353),
 ('tariff', 0.006425416112951842),
 ('contracts', 0.00595889730563687),
 ('customers', 0.005936881047453444)]

In [256]:
model.visualize_topics()

In [257]:
#new_topics, new_probs=model.reduce_topics(docs, topics, probs, nr_topics=20)

In [258]:
model.get_topic_info()

,Topic,Count,Name
0,-1,5844,-1_omni_message_please_would
1,0,486,0_enron_com_scott_mark
2,1,265,1_enron_com_dec_active
3,2,237,2_going_good_weekend_get
4,3,205,3_ect_hou_na_ees
...,...,...,...
141,140,11,140_google_notifications_dmml_account
142,141,11,141_jpg_aol_mail_hotmail
143,142,10,142_would_think_artwork_sheridan
144,143,10,143_bill_house_gramm_senate


In [259]:
model.get_topic(8)

[('recipient', 0.07137831663470377),
 ('intended', 0.06365701059548161),
 ('message', 0.055999305204572865),
 ('confidential', 0.03499542846614514),
 ('sender', 0.03142423227526155),
 ('privileged', 0.03039795607281623),
 ('original', 0.030213129169230753),
 ('mail', 0.029487689051655063),
 ('prohibited', 0.02774454337318103),
 ('contain', 0.02644453468501957)]

In [260]:
model.get_topic(15)

[('share', 0.16230649869214955),
 ('useful', 0.13749887665745147),
 ('handy', 0.11814666083057583),
 ('post', 0.11743823169614813),
 ('find', 0.1113066834347307),
 ('link', 0.11094236612525868),
 ('followers', 0.10780070286391774),
 ('gt', 0.09679954687099454),
 ('tweet', 0.09225361980943922),
 ('published', 0.08753982304663652)]

In [ ]:
topics_to_assign=[]
for item in gmail_data.iterrows():
    assigned_topic,prob=model.transform(item[1]['body'])
    topics_to_assign.append(assigned_topic[0])
gmail_data['Topic']=topics_to_assign

C:\Users\user\anaconda3\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
gmail_data

In [ ]:
gmail_data.to_csv('Clustered_Mails_BERTopic.csv',encoding='utf-8')

In [ ]:
model.save("Bert_model")